In [225]:
import ujson as json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.plotly as py

from moztelemetry.spark import get_pings
from moztelemetry import get_pings, get_pings_properties, get_one_ping_per_client
from datetime import date, timedelta
from __future__ import division

%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [226]:
sc.defaultParallelism

16

In [235]:
yesterday = (date.today() - timedelta(1)).strftime("%Y%m%d")
pings = get_pings(sc, app="Firefox", channel="release", submission_date=yesterday, fraction=.1)
pings.count()

655428

What keys are in a ping?

In [239]:
ping = json.loads(pings.first())
ping.keys()

[u'info',
 u'addonHistograms',
 u'slowSQL',
 u'ver',
 u'histograms',
 u'lateWrites',
 u'addonDetails',
 u'UIMeasurements',
 u'threadHangStats',
 u'simpleMeasurements',
 u'chromeHangs']

What's included in `addonDetails`?

In [240]:
ping['addonDetails']

{u'XPI': {u'gencrawler@some.com': {u'location': u'app-system-user',
   u'scan_MS': 3,
   u'scan_items': 8,
   u'unpacked': 1},
  u'webbooster@iminent.com': {u'location': u'winreg-app-global',
   u'scan_MS': 121,
   u'scan_items': 296,
   u'unpacked': 1},
  u'{3f963a5b-e555-4543-90e2-c3908898db71}': {u'location': u'winreg-app-global',
   u'scan_MS': 2,
   u'scan_items': 8,
   u'unpacked': 1},
  u'{972ce4c6-7e08-4474-a285-3208198ce6fd}': {u'location': u'app-global',
   u'scan_MS': 1,
   u'scan_items': 3,
   u'unpacked': 1}}}

We can get the list of enabled addons like so:

In [242]:
ping['addonDetails']['XPI']
ping['addonDetails']['XPI'].keys()

[u'gencrawler@some.com',
 u'{972ce4c6-7e08-4474-a285-3208198ce6fd}',
 u'{3f963a5b-e555-4543-90e2-c3908898db71}',
 u'webbooster@iminent.com']

For testing purposes, let's work with a sample:

In [243]:
sample = pings.sample(withReplacement=False, fraction=0.001)
sample.count()

654

Let's work with the following:

- Only check those pings with addon details
- Get a list of the addons per ping and tally them
- Combine tallies of all addons by key to get counts by addon


In [230]:
def hasAddonDetails(ping):
    return "addonDetails" in ping

def parse(ping):
    return json.loads(ping)
    
def getAddons(ping):
    return ping["addonDetails"]['XPI'].keys()

addon_list = sample.filter(hasAddonDetails).map(parse).map(getAddons)
addon_counts = addon_list.flatMap(lambda x: x).map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y).map(lambda x:(x[1],x[0])).sortByKey(ascending=False)

AttributeError: 'builtin_function_or_method' object has no attribute 'filter'

In [231]:
addon_counts.take(25)

KeyboardInterrupt: 